# Read csvs

## Import 

In [1]:
import os
import csv
import pandas as pd
import glob
import numpy as np

In [2]:
## set directory path
directoryPath = ('/Users/alia/Desktop/csvs/*.csv')

In [3]:
files = glob.glob(directoryPath)

In [4]:
#create a df
frame = pd.DataFrame()
list_ = []

## Read through csv files

In [5]:
for read in files:
    reader = pd.read_csv(read,index_col=None, header=0, encoding = "ISO-8859-1")
    patient = read[-7:-4]

    #Extract what you need into dataframe
    #	Need: pt ID		ErrorON		ErrorOFF	stimON		stimOFF		stimdur		stim latency	Error Lat. (this # will be negative, take only rows with - values)
    reader['during_stim_pos'] = reader['stim_onset'].astype(float) - reader['s_onset'].astype(float)
    reader['during_stim_neg'] = reader['s_onset'].astype(float) - reader['stim_offset'].astype(float)
    reader['stim_effect'] = reader['s_offset'].astype(float) - reader['stim_onset'].astype(float)
    reader['ptID'] = patient
    frame = frame.append(reader)

frame["during_stim_for_pos"] = np.where(frame["during_stim_pos"] > 0, 'yes', 'no')
frame["during_stim_for_neg"] = np.where(frame["during_stim_neg"] < 0, 'yes', 'no')

## export all data into a csv 

In [ ]:
#output column names in new csv file with extracted info
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","error2","region", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot"), axis=1)
# frame = frame.dropna()
#At the end of the for loop, write data to csv files (make sure to append)
frame.to_csv('all_patients.csv')

## export only trials with stim

In [ ]:
frame = frame[frame.stim == 'Y']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame.to_csv('stim_trials_only.csv')

## export everything without nopot

In [ ]:
frame = frame[frame.timePlot != 'noplot']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)
frame.to_csv('without_noplot_stim_only.csv')

## export only motor

In [ ]:
frame = frame[frame.error == 'motor']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)
frame.to_csv('motor_only.csv')

## export only word 

In [ ]:
frame = frame[frame.error == 'word']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame.to_csv('word_only.csv')

## Anatomy and Error Type

In [ ]:
##reindex frame 
frame1 = pd.DataFrame()
for i in frame:
    a = frame[i].tolist()
    frame1[i] = a

In [ ]:
import pandas as pd

In [ ]:
## find comorbid sites
patient = np.unique(frame1['ptID'])
# comorbid = pd.DataFrame()
comorbid_sites = []
co_index = []
for p in patient:
    motor = []
    word = []
    indx = np.where(frame1['ptID'] == p)
    for i in indx[0]:
        if frame1['error'][i] == 'motor':
            if frame1['stim_loc'][i] in motor: 
                pass
            else:
                motor.append(frame1['stim_loc'][i])
        if frame1['error'][i] == 'word':
            if frame1['stim_loc'][i] in word: 
                pass
            else:
                word.append(frame1['stim_loc'][i])
    for x in motor:
        if x in word:
            if str(x) == 'nan':
                pass
            else:
                comorbid_sites.append(str(x)+'_' + str(p))
print(comorbid_sites) 
print(co_index)
                


In [ ]:
sites = []
for c in co_index:
    sites.append(frame1['region'][c])

ventral_co = sites.count('ventral') 
dorsal_co = sites.count('ifg') 

print(sites)


In [ ]:
## counts how many errors are at a given site
def count(error,site):
    anatomy = pd.DataFrame()
    myidx = np.where(frame1['region'] == site)[0]
    errors = []
    for i in myidx:
        errors.append(frame1['error'][i])
        errors.append(frame1['error2'][i])
    counts = errors.count(error)
        
    return('number of ' + error + ' errors at site ' + site + ': ' + str(counts))

        

In [ ]:
## counts how many trials at a site (=errors and correct)
def count_trials(site):
    anatomy = pd.DataFrame()
    myidx = np.where(frame1['region'] == site)[0]
    region = []
    for i in myidx:
        region.append(frame1['region'][i])
    return(region.count(site))

In [ ]:
count('syllable','other')

In [ ]:
# framey

## Calculating Stim Gap

In [58]:
# framey = frame[frame.stim == 'Y']
# framey = framey[framey.timePlot != 'noplot']
# framey = framey[framey.error != '']

framey = frame.reindex_axis(("trial","ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","error2", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame1 = pd.DataFrame()
for i in framey:
    a = framey[i].tolist()
    frame1[i] = a

frame1['two_before_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['two_before_s_offset'] = np.zeros(len(frame1['s_onset']))
frame1['one_before_s_offset'] = np.zeros(len(frame1['s_onset']))
frame1['one_before_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['one_after_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['one_after_s_offset'] = np.zeros(len(frame1['s_onset']))


frame1['two_before_s_onset'][2:] = frame1['s_onset'][:-2]
frame1['two_before_s_offset'][2:] = frame1['s_offset'][:-2]

frame1['one_before_s_offset'][1:] = frame1['s_offset'][:-1]
frame1['one_before_s_onset'][1:] = frame1['s_onset'][:-1]

frame1['one_after_s_onset'][:-1] = frame1['s_onset'][1:]
frame1['one_after_s_offset'][:-1] = frame1['s_offset'][1:]




/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set o

In [13]:
frame1.to_csv('tester.csv')

In [59]:
## find timing 

frame1['tbon'] = frame1['two_before_s_onset'][2:].astype(float) - frame1['stim_onset'].astype(float)
frame1['tboff'] = frame1['two_before_s_offset'][2:].astype(float) - frame1['stim_onset'].astype(float)

frame1['obon'] = frame1['one_before_s_onset'][1:].astype(float) - frame1['stim_onset'].astype(float)
frame1['oboff'] = frame1['one_before_s_offset'][1:].astype(float) - frame1['stim_onset'].astype(float)

frame1['son'] = frame1['s_onset'].astype(float) - frame1['stim_onset'].astype(float)
frame1['soff'] = frame1['s_offset'].astype(float) - frame1['stim_onset'].astype(float)

frame1['oaon'] = frame1['one_after_s_onset'][:-1].astype(float) - frame1['stim_onset'].astype(float)
frame1['oaoff'] = frame1['one_after_s_offset'][:-1].astype(float) - frame1['stim_onset'].astype(float)

frame1['pt_response_time'] = frame1['stim_onset'] - frame1['s_onset']

frame1['before_gap'] = frame1['obon'] - frame1['tboff']

frame1['stim_gap'] = frame1['son'] - frame1['oboff']

frame1['after_gap'] = frame1['oaon'] - frame1['soff']









In [66]:
frame1 = frame1[frame1.timePlot != 'noplot']
frame1 = frame1[frame1.stim == 'Y']
frame1 = frame1.dropna(subset=['error'])
frame1 = frame1.reindex_axis(('trial','ptID','before_gap', 'stim_gap', 'after_gap', 'stim'), axis = 1)
# frame1['two_before_s_onset'] 
# frame1['timePlot']

In [ ]:
frame2 = frame1

In [ ]:
frame2 = frame2.reindex_axis(('one_before_s_onset', 'stim_onset'), axis = 1)

In [68]:
frame1.to_csv('stim_info.csv')

In [ ]:
frame1 = frame1[frame1.before_gap > 0]
frame1 = frame1[frame1.after_gap > 0]
frame1 = frame1[frame1.stim_gap > 0]

frame1 = frame1[frame1.before_gap < 10]
frame1 = frame1[frame1.after_gap < 10]
frame1 = frame1[frame1.stim_gap < 10]

In [ ]:
frame1.to_csv('stim_info.csv')

## Look at onset of stim - patient onset in ifg and precg

In [ ]:
frame_ifg = frame1[frame1.region == 'ifg']
frame_precg = frame1[frame1.region != 'ifg']
frame_precg = frame_precg[frame_precg.region != 'other']


frame_ifg = frame_ifg.reindex_axis(('region', 'pt_response_time'), axis = 1)
frame_precg = frame_precg.reindex_axis(('region', 'pt_response_time'), axis = 1)

In [ ]:
frame_ifg

##  Set column parameters and export csv

In [ ]:
framePlot = frame1.reindex_axis(("ptID","error", "tbon", "tboff", "oboff","son","soff","oaon","oaoff"), axis=1)
framePlot['error'].replace('', np.nan, inplace=True)
framePlot.dropna(subset=['error'], inplace=True)
print(framePlot) #to preview csv contents

framePlot.to_csv('stim_info_timingPlot.csv')